In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

import pathlib
import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="v40ZMYu9O9KrUmkE9j9O")
project = rf.workspace().project("lpcv_data")
dataset = project.version(2).download("yolov5")

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to LPCV_data-2 in yolov5pytorch:: 100%|██████████| 492/492 [00:00<00:00, 2942.31it/s]


# The next few cells can be run to download and train on the Kitti Dataset

In [7]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip -O kitti.zip

--2021-10-31 23:41:47--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.170.9
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.170.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12569945557 (12G) [application/zip]
Saving to: ‘kitti.zip’

kitti.zip           100%[===================>]  11.71G  21.0MB/s    in 9m 33s  

2021-10-31 23:51:21 (20.9 MB/s) - ‘kitti.zip’ saved [12569945557/12569945557]



In [8]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip -O kittilabel.zip

--2021-10-31 23:51:21--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.170.9
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.170.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5601213 (5.3M) [application/zip]
Saving to: ‘kittilabel.zip’

kittilabel.zip      100%[===================>]   5.34M  4.76MB/s    in 1.1s    

2021-10-31 23:51:23 (4.76 MB/s) - ‘kittilabel.zip’ saved [5601213/5601213]



In [9]:
with zipfile.ZipFile("./kitti.zip", 'r') as zip_ref:
    zip_ref.extractall("kitti")

import zipfile
with zipfile.ZipFile("./kittilabel.zip", 'r') as zip_ref:
    zip_ref.extractall("kittilabel")

In [10]:
data_dir = pathlib.Path('kitti/training/image_2')
label_dir = pathlib.Path('kitti/training/label_2')
yolo_labels = pathlib.Path('yolo')

In [11]:
image_count = len(list(data_dir.glob('*.png')))
label_count = len(list(label_dir.glob('*.txt')))

In [12]:
root_path = "./kitti/training/"
yolo_path = "yolo/"

In [13]:
kitti2yolotype_dict = {
    "Car": "0",
    "Van": "0",
    "Pedestrian": "1",
    "Person_sitting": "1",
    "Cyclist": "2",
    "Truck": "3",
    "Tram": "6",
    "Misc": "6",
    "DontCare": "6",
}
def kitti2yolo(kitti_label, img_height, img_width):

    kitti_label_arr = kitti_label.split(" ")
    x1 = float(kitti_label_arr[4])
    y1 = float(kitti_label_arr[5])
    x2 = float(kitti_label_arr[6])
    y2 = float(kitti_label_arr[7])

    bb_width = x2 - x1
    bb_height = y2 - y1
    yolo_x = (x1 + 0.5 * bb_width) / img_width
    yolo_y = (y1 + 0.5 * bb_height) / img_height
    yolo_bb_width = bb_width / img_width
    yolo_bb_height = bb_height / img_height
    yolo_label = kitti2yolotype_dict[kitti_label_arr[0]]

    return (
        yolo_label
        + " "
        + str(yolo_x)
        + " "
        + str(yolo_y)
        + " "
        + str(yolo_bb_width)
        + " "
        + str(yolo_bb_height)
    )

In [14]:
%cp -r kittilabel/training/label_2 kitti/training/label_2

In [15]:
kitti_images_path = root_path + "image_2" + os.sep
kitti_labels_path = root_path + "label_2" + os.sep

# yolo paths
if yolo_path is None:
    yolo_path = root_path + "yolo_labels" + os.sep

if not os.path.exists(yolo_path):
    os.makedirs(yolo_path)

# load each kitti label, convert to yolo and save
for labelfilename in os.listdir(kitti_labels_path):
  yolo_labels = []
  with open(kitti_labels_path + labelfilename, "r") as kittilabelfile:
    cvimage = cv2.imread(
        kitti_images_path + labelfilename.split(".txt")[0] + ".png"
        )
    height, width, frame_depth = cvimage.shape
    for kitti_label in kittilabelfile:
      yolo_labels.append(
          kitti2yolo(kitti_label, img_height=height, img_width=width)
          )
      with open(yolo_path + labelfilename, "w+") as yololabelfile:
        for label in yolo_labels:
          yololabelfile.write(label + "\n")

In [16]:
%mkdir kittiyolo
%cd kittiyolo
%mkdir train
%mkdir valid
%cd valid
%mkdir images
%mkdir labels
%cd ../..

/content/yolov5/kittiyolo
/content/yolov5/kittiyolo/valid
/content/yolov5


In [17]:
%cp -r yolo kittiyolo/train/labels
%cp -r kitti/training/image_2 kittiyolo/train/images

In [24]:
root_path = "./kittiyolo/train/"
kittiyolo_images_path = root_path + "images" + os.sep
kittiyolo_labels_path = root_path + "labels" + os.sep
yolo_valid_path = "./yolo/valid"

# load each kitti label, convert to yolo and save
count = 0
for labelfilename in os.listdir(kittiyolo_images_path):
  count = count+1
  if count == 3:
    break
  print(labelfilename)

006225.png
002322.png


In [25]:
src = "kittiyolo/train/images"
dst = "kittiyolo/valid/images"

# files = [i for i in os.listdir(src) if i.startswith("CTASK") and path.isfile(path.join(src, i))]

files = []
for i in os.listdir(src):
  x = str(i)[0:6]
  if(x[0]=='.'):
    continue
  inter = int(x)
  if inter<1000:
    files.append(i)

In [26]:
for f in files:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [27]:
src = "kittiyolo/train/labels"
dst = "kittiyolo/valid/labels"

# files = [i for i in os.listdir(src) if i.startswith("CTASK") and path.isfile(path.join(src, i))]

files = []
for i in os.listdir(src):
  x = str(i)[0:6]
  if(x[0]=='.'):
    continue
  inter = int(x)
  if inter<1000:
    files.append(i)


for f in files:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [ ]:
!python train.py --img 416 --batch 100 --epochs 10 --data ./kittidata.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=./data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=100, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-38-g7f9bbf0 torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

In [ ]:
!python detect.py --weights ./runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source test.jpg

detect: weights=['./runs/train/exp3/weights/best.pt'], source=pedestrian.jpeg, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-38-g7f9bbf0 torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 213 layers, 7031701 parameters, 0 gradients, 15.9 GFLOPs
image 1/1 /content/yolov5/pedestrian.jpeg: 448x640 10 Cars, 2 Vans, Done. (0.010s)
Speed: 0.5ms pre-process, 10.0ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2


# END OF KITTI DATASET

# The next few cells can be used to download and train on LPCV Data Directly
# You must upload the 5 people, 5 Ball M4V and CSV for this to work

In [ ]:
%mkdir LPCV
%cd LPCV
%mkdir train
%cd train
%mkdir images
%mkdir labels
%cd ..
%mkdir valid
%cd valid
%mkdir images
%mkdir labels
%cd ../..

/content/yolov5/LPCV
/content/yolov5/LPCV/test
/content/yolov5/LPCV
/content/yolov5/LPCV/valid
/content/yolov5


In [ ]:
vidcap = cv2.VideoCapture('5p5b_03A1.m4v')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("LPCV/train/images/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [ ]:
with open('5p5b_03A1.csv', newline='') as csvfile:
     csv_reader = csv.reader(csvfile)
     frame = 0
     yolo_labels = []
     for row in csv_reader:
       if row[0] == "Frame":
         continue
       if row[0] != frame:
         with open("LPCV/train/labels/frame" + str(frame) + ".txt", "w+") as yololabelfile:
          for label in yolo_labels:
            yololabelfile.write(label + "\n")
         frame = row[0]
         yolo_labels = []
       yoloarray = row
       del yoloarray[2:3]
       del yoloarray[0:1]
       yolorow = ' '.join(yoloarray)
       yolo_labels.append(yolorow)
     with open("LPCV/train/labels/frame" + str(frame) + ".txt", "w+") as yololabelfile:
       for label in yolo_labels:
         yololabelfile.write(label + "\n")

In [ ]:
src = "LPCV/train/labels"
dst = "LPCV/valid/labels"
src1 = "LPCV/train/images"
dst1 = "LPCV/valid/images"
random.seed(56)
rnum = random.randint(1, 10)

txtfiles = []
jpgfiles = []
for i in os.listdir(src):
  number = i[5:]
  if number[0] == 'b':
    continue
  number = number[:len(number)-4]
  inter = int(number)
  if inter%rnum==0:
    txtfiles.append(i)
    filestr = "frame"+number+".jpg"
    jpgfiles.append(filestr)


for f in txtfiles:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

for f in jpgfiles:
    shutil.copy(os.path.join(src1, f), dst1)
    p = os.path.join(src1, f)
    os.remove(p)

In [ ]:
!python train.py --img 416 --batch 100 --epochs 1 --data ./data2.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=./data2.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=1, batch_size=100, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-39-g5d4258f torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

In [ ]:
 !python detect.py --weights runs/train/exp5/weights/best.pt --img 640 --conf 0.25 --source 5p5b_03A1.m4v

detect: weights=['runs/train/exp5/weights/best.pt'], source=5p5b_03A1.m4v, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-39-g5d4258f torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/1859) /content/yolov5/5p5b_03A1.m4v: 384x640 Done. (0.017s)
video 1/1 (2/1859) /content/yolov5/5p5b_03A1.m4v: 384x640 Done. (0.012s)
video 1/1 (3/1859) /content/yolov5/5p5b_03A1.m4v: 384x640 Done. (0.012s)
video 1/1 (4/1859) /content/yolov5/5p5b_03A1.m4v: 384x640 Done. (0.013s)
video 1/1 (5/1859) /content/yolov5/5p5b_03A1.m4v: 384x640 Done. (0.012s)
video 1/1 (6/1859) /conte

# END OF LPCV DATA FROM WEBSITE

# The next cell can be used to train on our own LPCV dataset
# You must convert the LPCV_data-2 to LPCV_data_2 in the folder name and in data.yaml

In [2]:
!python train.py --img 416 --batch 10 --epochs 90 --data ./LPCV_data_2/data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=./LPCV_data_2/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=90, batch_size=10, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-40-g8c326a1 torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, transl

In [4]:
 !python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source 5p5b_03A1.mp4

detect: weights=['runs/train/exp/weights/best.pt'], source=5p5b_03A1.mp4, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-40-g8c326a1 torch 1.9.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 213 layers, 7037095 parameters, 0 gradients, 15.9 GFLOPs
video 1/1 (1/1859) /content/yolov5/5p5b_03A1.mp4: 384x640 1 Ball 1, 2 Ball 4s, 1 Ball 5, 1 Person 1, 1 Person 2, 1 Person 3, 1 Person 4, 3 Person 5s, Done. (0.019s)
video 1/1 (2/1859) /content/yolov5/5p5b_03A1.mp4: 384x640 1 Ball 1, 2 Ball 4s, 1 Ball 5, 1 Person 1, 1 Person 2, 1 Person 3, 1 Person 4, 3 Person 5s, Done. (0.011s)
video 1/1 (3/1859) /content/yolov5/5p5b_03A1.mp4: 384x640 1 B

In [41]:
!python export.py --weights runs/train/exp/weights/best.pt --include torchscript onnx coreml saved_model pb tflite

export: data=data/coco128.yaml, weights=runs/train/exp4/weights/best.pt, imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=13, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx', 'coreml', 'saved_model', 'pb', 'tflite']
YOLOv5 🚀 v6.0-39-g5d4258f torch 1.9.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7037095 parameters, 0 gradients, 15.9 GFLOPs

PyTorch: starting from runs/train/exp4/weights/best.pt (14.4 MB)

TorchScript: starting export with torch 1.9.0+cu111...
/content/yolov5/models/yolo.py:60: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4] or self.onnx_dynamic:
/co